In [1]:
import nltk
import wikipedia

In [2]:
def named_entity_recognition(text):
    data = nltk.word_tokenize(text)
    data = nltk.pos_tag(data)
    data = nltk.ne_chunk(data)
    data = list(data)
    
    final = False
    
    for x in data:
        if type(x) is nltk.tree.Tree:
            final = x[0][0]
            
    return final

In [3]:
d = named_entity_recognition("Can you give me some info about Cahrles")

In [4]:
wikipedia.search("Bulgaria")

['Bulgaria',
 'Demographics of Bulgaria',
 'Sofia',
 'Bulgaria Air',
 "People's Republic of Bulgaria",
 'Balkan Wars',
 'Volga Bulgaria',
 'List of Bulgarian consorts',
 'Georgi Markov',
 'Provinces of Bulgaria']

In [5]:
bg = wikipedia.page("Bulgaria")

In [7]:
" ".join(bg.categories)

'All articles containing potentially dated statements All articles needing additional references All articles with dead external links All articles with unsourced statements All articles with vague or ambiguous time Articles containing Bulgarian-language text Articles containing potentially dated statements from 2012 Articles including recorded pronunciations (English) Articles needing additional references from March 2018 Articles with Bulgarian-language external links Articles with Curlie links Articles with dead external links from December 2017 Articles with hAudio microformats Articles with permanently dead external links Articles with unsourced statements from October 2011 Balkan countries Bulgaria CS1 Bulgarian-language sources (bg) CS1 French-language sources (fr) CS1 German-language sources (de) Coordinates on Wikidata Former empires Good articles Member states of NATO Member states of the Council of Europe Member states of the European Union Member states of the Union for the

In [37]:
len(bg.content[:1*10**3])

1000

In [38]:
len(bg.content)

52454

In [24]:
# build_index.py

FILE_URL = "http://apps.sloanahrens.com/qbox-blog-resources/kaggle-titanic-data/test.csv"

ES_HOST = {
    "host" : "localhost", 
    "port" : 9300
}

INDEX_NAME = 'titanic'
TYPE_NAME = 'passenger'

ID_FIELD = 'passengerid'

import csv
import urllib.request as urllib2

from elasticsearch import Elasticsearch

response = urllib2.urlopen(FILE_URL)
csv_file_object = csv.reader(response)
 
header = csv_file_object.next()
header = [item.lower() for item in header]

bulk_data = [] 

for row in csv_file_object:
    data_dict = {}
    for i in range(len(row)):
        data_dict[header[i]] = row[i]
    op_dict = {
        "index": {
        	"_index": INDEX_NAME, 
        	"_type": TYPE_NAME, 
        	"_id": data_dict[ID_FIELD]
        }
    }
    bulk_data.append(op_dict)
    bulk_data.append(data_dict)

# create ES client, create index
es = Elasticsearch(hosts = [ES_HOST])

if es.indices.exists(INDEX_NAME):
    print("deleting '%s' index..." % (INDEX_NAME))
    res = es.indices.delete(index = INDEX_NAME)
    print(" response: '%s'" % (res))

request_body = {
    "settings" : {
        "number_of_shards": 1,
        "number_of_replicas": 0
    }
}

print("creating '%s' index..." % (INDEX_NAME))
res = es.indices.create(index = INDEX_NAME, body = request_body)
print(" response: '%s'" % (res))


# bulk index the data
print("bulk indexing...")
res = es.bulk(index = INDEX_NAME, body = bulk_data, refresh = True)
#print(" response: '%s'" % (res))


# sanity check
print("searching...")
res = es.search(index = INDEX_NAME, size=2, body={"query": {"match_all": {}}})
print(" response: '%s'" % (res))

print("results:")
for hit in res['hits']['hits']:
    print(hit["_source"])

URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>